<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

W tym notebooku przejdziemy całą ścieżkę - od uruchomienia modelu aż po zwrot danych w formacie JSON. Zaczynajmy! :)

### Przygotowanie środowiska

In [1]:
#%%capture

!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.9 MB/s eta 0:00:00


In [2]:
import gc
import torch

In [3]:
import warnings
warnings.filterwarnings("ignore") # tylko na potrzeby naszego demo ;)

### Inicjalizacja modelu (4 bit)

In [15]:
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, TextStreamer)

# Wybór urządzenia do uruchomienia modelu
device = "cuda"
model_name = 'speakleash/Bielik-7B-Instruct-v0.1'

# Ustawienie kwantyzacji na 4 bity, aby zmieścić na GPU na Colabie
load_4_bit = True

# Ustawienie tokenizera i kluczowych tokenów
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Obsługa streamingu
streamer = TextStreamer(tokenizer,
                        skip_prompt=True,
                        skip_special_tokens=True)

# Przygotowanie zmiennej pod konfigurację kwantyzacji
quantization_config = None

if load_4_bit:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config)



`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Funkcja pomocnicza do generowania odpowiedzi

In [43]:
# Parametry modelu
temperature = 0.0
max_tokens = 256
top_k = 100
top_p = 1

def generate(prompt: str,
             max_tokens: int,
             temperature: float = 0.0,
             system: str = None,
             return_str: bool = False):

  messages = []

  # Przygotowanie template z promptem
  if system:
    messages.append({"role": "system",
                     "content": system})

  messages.append({"role": "user",
                   "content": prompt})

  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
  model_inputs = input_ids

  # Przerzucenie inputu na GPU, jeśli jest dostępne
  if torch.cuda.is_available():
    model_inputs = input_ids.to(device)

  # Generowanie odpowiedzi przez model
  outputs = model.generate(
      model_inputs,
      streamer = streamer,
      max_new_tokens=max_tokens,
      do_sample=True if temperature else False,
      temperature = temperature,
      top_k = top_k,
      top_p = top_p)

  # Wydrukowanie odpowiedzi modelu bądź zwrotka do zmiennej
  if return_str:
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)

#### Test

In [12]:
# Test generowania tekstu przez model
generate("Hej, jesteś tam?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Cześć! Tak, jestem tutaj. W czym mogę Ci pomóc?


In [13]:
# Test z promptem systemowym i właściwym
generate(prompt="Hej, czym się zajmujesz?",
         system="Jesteś programistą baz danych, który chce rozwijać swoje kompetencje w obszarze AI")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Cześć! Jako programista baz danych specjalizuję się w projektowaniu, tworzeniu i zarządzaniu systemami, które przechowują i organizują duże zbiory danych. W ostatnim czasie zacząłem interesować się sztuczną inteligencją i uczeniem maszynowym, aby poszerzyć swoje kompetencje i zrozumieć, jak te technologie mogą wpływać na moją dziedzinę. Uczestniczę w kursach i czytam artykuły na temat AI, aby zdobyć wiedzę i umiejętności związane z wykorzystaniem AI w bazach danych. Chciałbym w przyszłości wdrażać rozwiązania AI i machine learning w mojej pracy, aby pomagać organizacjom w lepszym zrozumieniu i analizie ich danych. Czy masz jakieś pytania


# Przykład 1

Załóżmy, że chcesz dane osób w formacie JSON a jedyne co masz do dyspozycji to tekst gdzie są one zawarte. Jak ugryźć ten problem? Wystarczy użyć Bielika!

Wybrałem pierwszy z brzegu artykuł: https://wydarzenia.interia.pl/malopolskie/news-prezydent-tarnowa-z-zarzutami-grozi-mu-wiezienie,nId,7762425

Najpierw musimy wydobyć tekst z artykułu! Do tego przydadzą się pythonowe biblioteki **requests** oraz **justext**!

In [18]:
# Instalacja niezbędnych bibliotek
!pip install justext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 21.5 MB/s eta 0:00:00


In [41]:
import requests
import justext

url = "https://wydarzenia.interia.pl/malopolskie/news-prezydent-tarnowa-z-zarzutami-grozi-mu-wiezienie,nId,7762425"

def get_article_from_url(url: str):
  # Przygotowanie zmiennej do zbierania fragmentów artykułu ze strony internetowej
  article_parts = []

  # Zebranie odpowiedzi w HTML
  response = requests.get(url)

  # Przetworzenie odpowiedzi przy użyciu biblioteki justext
  paragraphs = justext.justext(response.content, justext.get_stoplist("Polish"))
  for paragraph in paragraphs:
    if not paragraph.is_boilerplate:
      article_parts.append(paragraph.text)

  return " ".join(article_parts)

In [42]:
# Test działania funkcji
get_article_from_url(url)

'Prezydent Tarnowa z zarzutami. Grozi mu więzienie Prezydent Tarnowa Jakub Kwaśny oraz były prorektor Uniwersytetu Ekonomicznego w Krakowie Piotr Buła usłyszeli zarzuty. To pokłosie śledztwa prowadzonego przez prokuraturę w sprawie nadużyć finansowych, do jakich miało dochodzić na krakowskiej uczelni. Podejrzani odmówili składania wyjaśnień. Obu grozi do ośmiu lat więzienia. Prezydent Tarnowa Jakub Kwaśny z zarzutami. Chodzi o nieprawidłowości na Uniwersytecie Ekonomicznym w Krakowie/Grzegorz Klatka/Artur Gawle/Agencja FORUM Reklama O sprawie zrobiło się głośno we wrześniu 2023 roku. Wówczas decyzją ówczesnego rektora Uniwersytetu Ekonomicznego w Krakowie prof. Stanisława Mazura funkcję prorektora ds. współpracy stracił prof. Piotr Buła, a ze stanowiska dyrektora Krakowskiej Szkoły Biznesu UEK odwołany został dr Jakub Kwaśny. Prezydent Tranowa Jakub Kwaśny usłyszał zarzuty. Chodzi o nieprawidłowości na uczelni Z pojawiających się wtedy doniesień wynikało, że chodzi o możliwe nieprawidł

In [44]:
# Pobranie treści artykułu do zmiennej
article = get_article_from_url(url)

# Przygotowanie promptów
system = 'Jesteś ekspertem od ekstrakcji danych osobowych z artykułów i strukturyzowania ich do formatu obiektów JSON. Przeanalizuj otrzymany tekst i zwróć odpowiedź z formacie JSON z kluczami takimi jak: imię, nazwisko, funkcja, miejsce zamieszkania.'
prompt = f"Tekst do analizy: {article}"

# Zwiększenie limitu tokenów
max_tokens = 500

# Generowanie odpowiedzi Bielika!
generate(prompt,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Prezydent Tarnowa Jakub Kwaśny oraz były prorektor Uniwersytetu Ekonomicznego w Krakowie Piotr Buła usłyszeli zarzuty w związku z nadużyciami finansowymi na uczelni. Obaj podejrzani odmówili składania wyjaśnień. Grozi im do ośmiu lat więzienia. Sprawa ma kontekst wyborczy, ponieważ prof. Piotr Buła planował startować w wyborach na rektora UEK, a Jakub Kwaśny jest obecnie prezydentem Tarnowa. Obaj twierdzą, że są niewinni i wytoczyli sprawę o zniesławienie przeciwko byłemu rektorowi UEK Stanisławowi Mazurowi.


### Odpowiedź (na ten moment z Areny)

Na podstawie podanego tekstu, stworzyłem obiekt JSON zawierający wymagane informacje:

```json
{
  "osoby": [
    {
      "imie": "Jakub",
      "nazwisko": "Kwaśny",
      "funkcja": "Prezydent Tarnowa",
      "miejsce_zamieszkania": "Tarnów"
    },
    {
      "imie": "Piotr",
      "nazwisko": "Buła",
      "funkcja": "Były prorektor Uniwersytetu Ekonomicznego w Krakowie",
      "miejsce_zamieszkania": "Kraków"
    }
  ]
}
```
Wyjaśnienie krok po kroku:

1. Zidentyfikowałem kluczowe informacje o osobach wymienionych w tekście:
* Jakub Kwaśny: Prezydent Tarnowa
* Piotr Buła: Były prorektor Uniwersytetu Ekonomicznego w Krakowie

2. Określiłem miejsce zamieszkania na podstawie kontekstu:
* Jakub Kwaśny: Tarnów (jako prezydent miasta)
* Piotr Buła: Kraków (związany z Uniwersytetem Ekonomicznym w Krakowie)

3. Utworzyłem obiekt JSON z listą osób, gdzie każda osoba ma przypisane imię, nazwisko, funkcję i miejsce zamieszkania.

# Przykład 2

Wykorzystamy ten sam artykuł, ale damy modelowi nieco większą swobodę twórczą w prompcie!

In [45]:
article = get_article_from_url(url)

system = 'Jesteś ekspertem od ekstrakcji danych osobowych z artykułów i strukturyzowania ich do formatu obiektów JSON. Przeanalizuj otrzymany tekst i zwróć odpowiedź z formacie JSON z istotnymi informacjami, na które warto zwrócić uwagę.'
prompt = f"Tekst do analizy: {article}"
max_tokens = 550

generate(prompt,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Prezydent Tarnowa Jakub Kwaśny oraz były prorektor Uniwersytetu Ekonomicznego w Krakowie Piotr Buła usłyszeli zarzuty w związku z nadużyciami finansowymi na uczelni. Obaj podejrzani odmówili składania wyjaśnień. Grozi im do ośmiu lat więzienia. Sprawa ma kontekst wyborczy, ponieważ prof. Piotr Buła planował startować w wyborach na rektora UEK, a Jakub Kwaśny jest obecnie prezydentem Tarnowa. Obaj twierdzą, że są niewinni i wytoczyli sprawę o zniesławienie przeciwko byłemu rektorowi UEK Stanisławowi Mazurowi.


### Odpowiedź Bielika (z Areny)

Oto informacje z podanego tekstu w formacie JSON:

```json
{
  "osoby": [
    {
      "imie": "Jakub",
      "nazwisko": "Kwaśny",
      "stanowisko": "Prezydent Tarnowa",
      "zarzuty": ["nadużycia finansowe", "nieprawidłowości w wydatkowaniu środków publicznych", "nadużycie uprawnień"],
      "kara": "do 8 lat więzienia",
      "reakcja": "nie przyznał się do zarzutów, odmówił składania wyjaśnień",
      "oswiadczenie": "nie ma sobie nic do zarzucenia"
    },
    {
      "imie": "Piotr",
      "nazwisko": "Buła",
      "stanowisko": "były prorektor Uniwersytetu Ekonomicznego w Krakowie",
      "zarzuty": ["nadużycia finansowe", "nieprawidłowości w wydatkowaniu środków publicznych", "nadużycie uprawnień"],
      "kara": "do 8 lat więzienia",
      "reakcja": "nie przyznał się do zarzutów, odmówił składania wyjaśnień",
      "oswiadczenie": "zwrócił uwagę na kontekst wyborczy sprawy"
    },
    {
      "imie": "Stanisław",
      "nazwisko": "Mazur",
      "stanowisko": "były rektor Uniwersytetu Ekonomicznego w Krakowie, obecnie wiceprezydent Krakowa",
      "powiazania": ["wytoczona sprawa o zniesławienie przez Kwaśnego i Bułę"]
    }
  ],
  "sprawa": {
    "opis": "nadużycia finansowe na Uniwersytecie Ekonomicznym w Krakowie",
    "kwota": "ponad 219 tys. złotych",
    "zarzuty": ["nadużycie uprawnień", "działanie w celu osiągnięcia korzyści majątkowej"],
    "prokuratura": {
      "postawienie_zarzutow": "wrzesień 2023",
      "przedluzenie_sledztwa": "do 16 listopada"
    },
    "media": {
      "lokalne": "lovekrakow.pl",
      "inne": ["lokalne media"]
    }
  },
  "kontekst": {
    "wybory": {
      "rektora_UEK": {
        "kandydaci": ["Piotr Buła"],
        "przeszkody": ["decyzje rady uczelni"]
      }
    }
  }
}
```

Ten format JSON zawiera kluczowe informacje z podanego tekstu, pogrupowane w logiczne sekcje. Każda osoba ma swoje podsekcje z istotnymi szczegółami, a sprawa i kontekst są oddzielnie opisane. Format ten ułatwia dalsze przetwarzanie i analizę danych.

# Przykład 3

Teraz Bielik dostanie pełną swobodę, zobaczymy co odpowie tym razem...

In [46]:
article = get_article_from_url(url)

system = 'Jesteś ekspertem od ekstrakcji danych osobowych z artykułów i strukturyzowania ich do formatu obiektów JSON. Przeanalizuj otrzymany tekst i zwróć go w formacie JSON.'
prompt = f"Tekst do analizy: {article}"
max_tokens = 550

generate(prompt,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Prezydent Tarnowa Jakub Kwaśny oraz były prorektor Uniwersytetu Ekonomicznego w Krakowie Piotr Buła usłyszeli zarzuty w związku z nadużyciami finansowymi na uczelni. Obaj podejrzani odmówili składania wyjaśnień. Grozi im do ośmiu lat więzienia. Sprawa ma kontekst wyborczy, ponieważ prof. Piotr Buła planował startować w wyborach na rektora UEK, a Jakub Kwaśny jest obecnie prezydentem Tarnowa. Obaj twierdzą, że są niewinni i wytoczyli sprawę o zniesławienie przeciwko byłemu rektorowi UEK Stanisławowi Mazurowi.


Oto strukturyzowane dane w formacie JSON na podstawie podanego tekstu:

```json
{
  "tytul": "Prezydent Tarnowa z zarzutami. Grozi mu więzienie",
  "osoby": [
    {
      "imie": "Jakub",
      "nazwisko": "Kwaśny",
      "stanowisko": "Prezydent Tarnowa",
      "zarzuty": ["nadużycia finansowe", "nieprawidłowości w wydatkowaniu środków publicznych", "nadużycie uprawnień"],
      "kara": "do 8 lat więzienia",
      "reakcja": "nie przyznał się do zarzutów, odmówił składania wyjaśnień",
      "oswiadczenie": "nie ma sobie nic do zarzucenia"
    },
    {
      "imie": "Piotr",
      "nazwisko": "Buła",
      "stanowisko": "były prorektor Uniwersytetu Ekonomicznego w Krakowie",
      "zarzuty": ["nadużycia finansowe", "nieprawidłowości w wydatkowaniu środków publicznych", "nadużycie uprawnień"],
      "kara": "do 8 lat więzienia",
      "reakcja": "nie przyznał się do zarzutów, odmówił składania wyjaśnień",
      "oswiadczenie": "zwrócił uwagę na kontekst wyborczy sprawy"
    },
    {
      "imie": "Stanisław",
      "nazwisko": "Mazur",
      "stanowisko": "były rektor Uniwersytetu Ekonomicznego w Krakowie, obecnie wiceprezydent Krakowa",
      "powiazania": ["wytoczona sprawa o zniesławienie przez Kwaśnego i Bułę"]
    }
  ],
  "uczelnia": {
    "nazwa": "Uniwersytet Ekonomiczny w Krakowie",
    "szkola_biznesu": {
      "nazwa": "Krakowska Szkoła Biznesu UEK",
      "nieprawidlowosci": "wydatkowanie środków publicznych i zawieranie umów cywilnoprawnych",
      "kwota": "ponad 219 tys. złotych"
    }
  },
  "data": "wrzesień 2023",
  "postepowanie": {
    "status": "w toku",
    "przedluzenie": "do 16 listopada"
  },
  "media": {
    "lokalne": ["lovekrakow.pl"]
  }
}
```

Ten format JSON zawiera kluczowe informacje z tekstu, zorganizowane w hierarchiczny sposób, co ułatwia dostęp do danych i ich analizę.

## Podsumowanie
Jak widać, ekstrakcja danych do formatu JSON z tekstu jest dziecinnie prosta dla Bielika. Wystarczy krótki, prosty prompt, darmowe GPU na Colabie i voila - dane w JSONie gotowe!

Zapraszam do sprawdzenia pozostałych notebooków i tutoriali przygotowanych przez naszą społeczność!